# import

In [1]:
import pandas as pd

In [ ]:
file = 'data/bundesliga'

In [4]:
file = '../data/result'

In [5]:
df = pd.read_csv(file + '.csv', parse_dates=['eventbegin'])

In [6]:
df.head()

,programevent_id,name_de,eventbegin,channel,market,turnover
0,190001810,Bayern München - Bayer Leverkusen,2017-08-18 20:38:08,Online,goal-scorer-halftime,NaN
1,190001810,Bayern München - Bayer Leverkusen,2017-08-18 20:38:08,Retail,team-points-more-less,NaN
2,190001810,Bayern München - Bayer Leverkusen,2017-08-18 20:38:08,Retail,standard,NaN
3,190001810,Bayern München - Bayer Leverkusen,2017-08-18 20:38:08,Retail,handicap,NaN
4,190001810,Bayern München - Bayer Leverkusen,2017-08-18 20:38:08,Online,points-more-less-than,NaN


# feature engineering

In [7]:
def get_features(df):
    # add day of week
    df['weekday'] = df['eventbegin'].dt.weekday_name

    # add period of day
    df = df.assign(
        period=pd.cut(
            x=df['eventbegin'].dt.hour,
            bins=[-1, 12, 17, 24],
            labels=['Morning', 'Afternoon', 'Evening']
        )
    )

    # split datetime
    new_dates, new_times = zip(*[(d.date(), d.time()) for d in df['eventbegin']])
    df = df.assign(date=new_dates, time=new_times).drop(labels=['eventbegin'], axis=1)

    # split participants
    df['participant_a'], df['participant_b'] = df['name_de'].str.strip().str.split(pat=' - ', n=1).str
    df = df.drop(labels=['name_de'], axis=1)
    
    return df

In [8]:
df = get_features(df)

In [9]:
df.head()

,programevent_id,channel,market,turnover,weekday,period,date,time,participant_a,participant_b
0,190001810,Online,goal-scorer-halftime,NaN,Friday,Evening,2017-08-18,20:38:08,Bayern München,Bayer Leverkusen
1,190001810,Retail,team-points-more-less,NaN,Friday,Evening,2017-08-18,20:38:08,Bayern München,Bayer Leverkusen
2,190001810,Retail,standard,NaN,Friday,Evening,2017-08-18,20:38:08,Bayern München,Bayer Leverkusen
3,190001810,Retail,handicap,NaN,Friday,Evening,2017-08-18,20:38:08,Bayern München,Bayer Leverkusen
4,190001810,Online,points-more-less-than,NaN,Friday,Evening,2017-08-18,20:38:08,Bayern München,Bayer Leverkusen


# tidying

In [10]:
from difflib import SequenceMatcher

class SequenceCorrector:
    """Iterator for looping over a sequence backwards."""
    def __init__(self, s):
        self.s = s
        self.s_unique = s.unique()

    def similar(self, a, b):
        return SequenceMatcher(None, a, b).ratio()
        
    def correct_duplicates(self):
        for a in self.s_unique:
            for b in self.s_unique:
                result = self.similar(a, b)
                if result > .6 and result < 1.0:
                    print('to_replace =', a, ':', b, 'similarity =', result)
                    self.s = self.s.replace(to_replace={a:b})

        return self.s

In [11]:
sc = SequenceCorrector(df['participant_a'])
df['participant_a'] = sc.correct_duplicates()

to_replace = Borussia M'gladbach : Bor. M'gladbach similarity = 0.8235294117647058
to_replace = Eint. Frankfurt : Eintracht Frankfurt similarity = 0.8235294117647058
to_replace = Augsburg : FC Augsburg similarity = 0.8421052631578947
to_replace = Borussia Dortmund : Bor. Dortmund similarity = 0.8
to_replace = FC Augsburg : Augsburg similarity = 0.8421052631578947
to_replace = Eintracht Frankfurt : Eint. Frankfurt similarity = 0.8235294117647058
to_replace = Bor. Dortmund : Borussia Dortmund similarity = 0.8
to_replace = Bor. M'gladbach : Borussia M'gladbach similarity = 0.8235294117647058


In [12]:
sc = SequenceCorrector(df['participant_b'])
df['participant_b'] = sc.correct_duplicates()

to_replace = Wolfsburg : VfL Wolfsburg similarity = 0.8181818181818182
to_replace = B. M'gladbach : Borussia M'gladbach similarity = 0.75
to_replace = VfL Wolfsburg : Wolfsburg similarity = 0.8181818181818182
to_replace = Borussia M'gladbach : B. M'gladbach similarity = 0.75


In [13]:
df['participant_a'].unique()

array(['Bayern München', '1899 Hoffenheim', 'Hamburger SV',
       'VfL Wolfsburg', 'Hertha BSC', 'FSV Mainz 05', 'Schalke 04',
       'SC Freiburg ', "Borussia M'gladbach", '1.FC Köln',
       'Werder Bremen', 'Eint. Frankfurt', 'Augsburg', 'VfB Stuttgart',
       'Bayer Leverkusen', 'Borussia Dortmund', 'RB Leipzig',
       'Hannover 96'], dtype=object)

In [14]:
df['participant_b'].unique()

array(['Bayer Leverkusen', 'Werder Bremen', 'FC Augsburg',
       'Borussia Dortmund', 'VfB Stuttgart', 'Hannover 96', 'RB Leipzig',
       'Eintracht Frankfurt', '1.FC Köln', 'Hamburger SV',
       'Bayern München', 'Wolfsburg', "B. M'gladbach", 'FSV Mainz 05',
       '1899 Hoffenheim', 'Hertha BSC', 'SC Freiburg', 'Schalke 04'],
      dtype=object)

In [15]:
df.head()

,programevent_id,channel,market,turnover,weekday,period,date,time,participant_a,participant_b
0,190001810,Online,goal-scorer-halftime,NaN,Friday,Evening,2017-08-18,20:38:08,Bayern München,Bayer Leverkusen
1,190001810,Retail,team-points-more-less,NaN,Friday,Evening,2017-08-18,20:38:08,Bayern München,Bayer Leverkusen
2,190001810,Retail,standard,NaN,Friday,Evening,2017-08-18,20:38:08,Bayern München,Bayer Leverkusen
3,190001810,Retail,handicap,NaN,Friday,Evening,2017-08-18,20:38:08,Bayern München,Bayer Leverkusen
4,190001810,Online,points-more-less-than,NaN,Friday,Evening,2017-08-18,20:38:08,Bayern München,Bayer Leverkusen


In [16]:
df.columns

Index(['programevent_id', 'channel', 'market', 'turnover', 'weekday', 'period',
       'date', 'time', 'participant_a', 'participant_b'],
      dtype='object')

In [17]:
df.to_parquet(fname=file + '.parquet', engine='pyarrow')